<a href="https://colab.research.google.com/github/ngngocsonan2610/note/blob/master/1.%20Data%20Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Sources](#scrollTo=BvORvDCvAdH6)

>[1.EDA](#scrollTo=ltyLBhWQm2YF)

>>[Resume & Phân tích df](#scrollTo=pmXBVoIJDDjN)

>>[Correlation](#scrollTo=DEhCkATJnX0l)

>>[Statistical Checking](#scrollTo=TdqQPJ9zuPAh)

>>[Bayesian Statistic](#scrollTo=foaLMVH2wNDH)

>[2.Cleaning Data](#scrollTo=arothx_v_lC-)

>>[Missing value](#scrollTo=FSzLvImTnBdI)

>>[Outliers](#scrollTo=G9dkPpmLmkDE)

>[3.Features Processing](#scrollTo=81RBd7sRk0fu)

>>[string processing /NLP processing](#scrollTo=OkQAo4CboTLF)

>>[merge/concat](#scrollTo=pEwO7OIEmZ-B)

>>[where](#scrollTo=0ZwKNHDtmQ4h)

>>[group by](#scrollTo=GZ-VCSnElcgU)

>>[Binning](#scrollTo=OcFpKNKUnEoX)

>>[Time](#scrollTo=yWm9sbjolESN)

>[4.Visualization](#scrollTo=zJUeNk50VTOP)

>>[Note](#scrollTo=wuLwfvj3LvGi)

>>[Plot continous features](#scrollTo=OUdukOL-nnMp)

>>[Plot categorical](#scrollTo=QcyJ6xFLmJqM)



# Sources

In [0]:
%%html
<iframe src="https://stackedit.io/app#" width="800" height="600"></iframe>

- [**A Comprehensive Guide to Data Exploration**](https://www.analyticsvidhya.com/blog/2016/01/guide-data-exploration/)
- [Data Analytics Collection - Hasbrain](https://note.hasbrain.com/5c875913b40548001824aa67?target=path)


# 1.EDA 

## Resume & Phân tích df
Descriptive statistics are used to describe the basic features of the data in a study. They provide simple summaries about the sample and the measures. Together with simple graphics analysis, they form the basis of virtually every quantitative analysis of data.
- Phân tích về min max, mean, median, mode, std
- Phân tích về loại của feature (continous/categorical, numeric/text)
- Đếm giá trị unique ở features/ giá trị missing 

**Sources**:
- [Đọc thêm 1](https://towardsdatascience.com/understanding-descriptive-statistics-c9c2b0641291)



In [0]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

WOE_numeric_col = [x for x in df_WOE.select_dtypes(include=numerics).columns
              if x not in label+ ag_id+ time
              ]
WOE_cat_col = [x for x in df_WOE.columns 
           if x not in WOE_numeric_col + label + ag_id + time]

ratio_col = [x for x in WOE_numeric_col if 'ratio' in x]

print(' WOE_numeric_col: ', WOE_numeric_col)
print('\n WOE_cat_col: ', WOE_cat_col)
print('\n ratio_col: ', ratio_col)

In [0]:
# This function assits to expand Exploratory Data Analysis (EDA) is an open-ended 
#process where we calculate statistics and make figures to find trends, anomalies,
#patterns, or relationships within the data. The goal of EDA is to learn what our
#data can tell us. It generally starts out with a high level overview, 
#then narrows in to specific areas as we find intriguing areas of the data. 

def des_stat_analyze(df_input):
    # check number of rows, cols
    no_rows = df_input.shape[0]
    no_cols = df_input.shape[1]
    print("No. observations:", no_rows )
    print("No. features:", no_cols )
  
    # checking type of features
    name = []
    cols_type = []
    for n,t in df_input.dtypes.iteritems():
        name.append(n)
        cols_type.append(t)

    # checking distinction (unique values) of features
    ls_unique = []
    for cname in df_input.columns:
        try:
            nunique = df_input[cname].nunique()
            pct_unique = nunique*100.0/ no_rows
            ls_unique.append("{} ({:0.2f}%)".format(nunique, pct_unique))
        except:
            ls_unique.append("{} ({:0.2f}%)".format(0,0))
            continue

    # checking missing values of features
    ls_miss = []
    for cname in df_input.columns:
        try:
            nmiss = df_input[cname].isnull().sum()
            pct_miss = nmiss*100.0/ no_rows
            ls_miss.append("{} ({:0.2f}%)".format(nmiss, pct_miss))
        except:
            ls_miss.append("{} ({:0.2f}%)".format(0,0))
            continue 
      
    # checking zeros
    ls_zeros = []
    for cname in df_input.columns:
        try:
            nzeros = (df_input[cname] == 0).sum()
            pct_zeros = nzeros * 100.0/ no_rows
            ls_zeros.append("{} ({:0.2f}%)".fornat(nzeros, pct_zeros))
        except:
            ls_zeros.append("{} ({:0.2f}%)".format(0,0))
            continue
      
    # checking negative values
    ls_neg = []
    for cname in df_input.columns:
        try:
            nneg = (df_input[cname].astype("float")<0).sum()
            pct_neg =nneg * 100.0 / no_rows
            ls_neg.append("{} ({:0.2f}%)".format(nneg, pct_neg))
        except:
            ls_neg.append("{} ({:0.2f}%)".format(0,0))
            continue
      
    # extracting the output
    data = {
      "name": name,
      "col_type": cols_type,
      "n_unique": ls_unique,
      "n_miss": ls_miss,
      "n_zeros":ls_zeros,
      "n_neg":ls_neg      
    }
  
    # statistical info
    df_stats = df_input.describe().transpose()
    ex_stats = pd.concat([df_input.median(),df_input.kurtosis(),df_input.skew()], axis = 1)
    ex_stats = ex_stats.rename(columns={0:"median",1:"kurtosis",2:"skew"})
    df_stats = df_stats.merge(ex_stats, left_index=True, right_index=True)
    #ls_stats = []
    for stat in df_stats.columns:
        data[stat] = []
        for cname in df_input.columns:
            try:
                data[stat].append(df_stats.loc[cname, stat])
            except:
                data[stat].append("NaN")
        
    # take samples
    df_sample = df_input.sample(frac = .5).head().transpose()
    df_sample.columns = ["sample_{}".format(i) for i in range(5)]
  
    # repair the output
    col_ordered = ["name","col_type","count","n_unique","n_miss","n_zeros","n_neg",
                "25%","50%","75%","max","min","mean","median","std","kurtosis","skew"]
    df_data = pd.DataFrame(data, columns = col_ordered).set_index("name")
    df_data = pd.concat([df_data, df_sample], axis = 1)

    return df_data.reset_index().sort_values(by=['col_type'])  

In [0]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['Max'] = df.max().values
    summary['Min'] = df.min().values
    summary['Mean'] = df.mean().values
    summary['Var'] = df.std().values
    summary['Q1'] = df.quantile(0.25).values
    summary['Q3'] = df.quantile(0.75).values    
    
    
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values
#     summary['Sample 1'] = df.sample(1).values
#     summary['Sample 2'] = df.sample(1).values
#     summary['Sample 3'] = df.sample(1).values
    

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 
    summary.sort_values(by=['Name', 'dtypes'])
    return summary

In [0]:
'''
Tìm các bất ổn về missing value và các cột có bật ổn về số lượng giá trị
'''
many_null_cols = [col for col in df_train.columns if df_train[col].isnull().sum() / df_train.shape[0] > 0.9]
many_null_cols_test = [col for col in df_test.columns if df_test[col].isnull().sum() / df_test.shape[0] > 0.9]


big_top_value_cols = ([col for col in train.columns if 
                       df_train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9])
big_top_value_cols_test = ([col for col in test.columns if 
                            df_test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9])
one_value_cols = [col for col in df.columns if df[col].nunique() <= 1]
one_value_cols_test = [col for col in df_test.columns if df_test[col].nunique() <= 1]
one_value_cols == one_value_cols_test

cols_to_drop = list(set(many_null_cols + many_null_cols_test + 
                        big_top_value_cols + big_top_value_cols_test + 
                        one_value_cols + one_value_cols_test))
cols_to_drop.remove('isFraud')
len(cols_to_drop)

df = df.drop(cols_to_drop, axis=1)



## Correlation

Xem correlation của các features với nhau: 
- 2 features có chỉ số correlation dương, có nghĩa khi A tăng thì B tăng (if A increase, B will increase)
- 2 features có chỉ số correlation âm, có nghĩa khi A tăng thì B giảm (if A increase, B will decrease)
- - 2 features có chỉ số correlation cao, có nghĩa là 2 features này có khả năng thay thế nhau trong model (replaces)

Sources:
-  [Covariance vs Correlation](https://forum.machinelearningcoban.com/t/covariance-va-correlation/767)

In [0]:
def plot_corr(df):
    correlations = df.corr()

    # Using seaborn package
    # Generate a mask for the upper triangle
    mask = np.zeros_like(correlations, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(16, 16))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(260, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(correlations, mask=mask, cmap=cmap, vmin = -1, vmax= 1 , center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.yticks(rotation=0)

In [0]:
def find_topcorr_bycol(col):
    df_corr = df.corr().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
    df_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}
                , inplace=True)     
    return df_corr[df_corr["Feature 1"]==col]

## Statistical Checking
Các kỹ thuật khác trong việc kiểm tra về mặt thống kê

Source:
- [Statistical Learning oxh](https://ongxuanhong.wordpress.com/category/kien-thuc/statistical-inference/)
- [Video về Statistic](https://www.youtube.com/user/drnguyenvtuan/videos)

## Bayesian Statistic
- [How Bayesian statistics convinced me to hit the gym](https://towardsdatascience.com/how-bayesian-statistics-convinced-me-to-hit-the-gym-fa737b0a7ac)
- [Think you need to learn Bayesian Analysis? Read this first](https://peadarcoyle.com/2019/01/01/think-you-need-to-learn-bayesian-analysis-read-this-first/?fbclid=IwAR1KGEuQNfzzf0BexNampJsyZ3zNgcfzDOgNMsPvppjElAu4qOOPF8AwC7g)



# 2.Cleaning Data

Used in:
  - Kaggle:[ Fraud Detection](https://www.kaggle.com/sonannguyenngoc/fraud-pca/edit/run/18581236) 

## Nan/Inf value
- fill na : https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779

- 


In [0]:
# Tìm na/nan/null trong từng cột
# Hiện thị: Số na, %na, type
# GPreda, missing data
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

df['Count_miss'] = df[col].isnull().sum(axis=1)

df['nulls1'] = df.isna().sum(axis=1)

In [0]:
'''
1 số cách fill NA
interpolate from categorical features
'''


df = df[df['MEMBER_GENDER'].notnull()]

df['MEMBER_MARITAL_STATUS'] = df['MEMBER_MARITAL_STATUS'].fillna('UNK')
df['MEMBER_MARITAL_STATUS'] = df['MEMBER_MARITAL_STATUS'].replace({'S':'S-W-D','W':'S-W-D','D':'S-W-D'})


df['MEMBER_ANNUAL_INCOME'] = df.groupby(['MEMBER_OCCUPATION_CD','MEMBER_MARITAL_STATUS','MEMBER_GENDER'])
                              \ ['MEMBER_ANNUAL_INCOME'].apply(lambda x: x.fillna(x.median()))  

titanic_all.groupby(['Pclass', 'Parch', 'SibSp']).Fare.median()[3][0][0]

titanic_all['Fare'] = titanic_all['Fare'].fillna(titanic_all.groupby(
    ['Pclass', 'Parch', 'SibSp']).Fare.transform('mean'))

In [0]:
np.isinf(df[ratio_col]).sum()

In [0]:
'''
đếm số lượng bằng 0 cho mỗi dòng trên toàn bộ data
'''

t1['count_zero'] = (t1 == 0).astype(int).sum(axis=1)

In [0]:
df.groupby('name')['activity'].value_counts().unstack().fillna(0)

## Outliers

[estimation cdf - plot](https://www.jddata22.com/home//plotting-an-empirical-cdf-in-python)
  - check outlier by std and mean
  
[boxplot & quanlite](https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51)
  - check outlier by 
    - IQR: interquantile range = Q3 - Q1
    - “maximum”: Q3 + 1.5*IQR
    - “minimum”: Q1 -1.5*IQR

z-score


**News**
- [Ways to Detect and Remove the Outliers](https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba)
- [Three ways to detect outliers](http://colingorrie.github.io/outlier-detection.html)
- [How to Use Statistics to Identify Outliers in Data](https://machinelearningmastery.com/how-to-use-statistics-to-identify-outliers-in-data/)
- [5 Ways to Find Outliers in Your Data](https://statisticsbyjim.com/basics/outliers/)

In [0]:
def CalcOutliers(df_num): 
    '''
    Find Outliers for continous cols
    Tìm outliner dựa trên std, và mean

    Sau khi phân tích quantile và mean, std
    có thể biết outlier nằm ngoài vùng nào:
    vd: cut = std * 3
    '''
    # calculating mean and std of the array
    data_mean, data_std = np.mean(df_num), np.std(df_num)

    # seting the cut line to both higher and lower values
    # You can change this value
    cut = data_std * 3

    #Calculating the higher and lower cut values
    lower, upper = data_mean - cut, data_mean + cut

    # creating an array of lower, higher and total outlier values 
    outliers_lower = [x for x in df_num if x < lower]
    outliers_higher = [x for x in df_num if x > upper]
    outliers_total = [x for x in df_num if x < lower or x > upper]

    # array without outlier values
    outliers_removed = [x for x in df_num if x > lower and x < upper]
    
    print('Identified lowest outliers: %d' % len(outliers_lower)) # printing total number of values in lower cut of outliers
    print('Identified upper outliers: %d' % len(outliers_higher)) # printing total number of values in higher cut of outliers
    print('Total outlier observations: %d' % len(outliers_total)) # printing total number of values outliers of both sides
    print('Non-outlier observations: %d' % len(outliers_removed)) # printing total number of non outlier values
    print("Total percentual of Outliers: ", round((len(outliers_total) / len(outliers_removed) )*100, 4)) # Percentual of outliers in points
    
    return

In [0]:
'''
which defined as the proportion of the IQR past the low and high quartiles to extend the plot whiskers 
or interquartile range (IQR)
therefore, maximum = Q3 + 1.5*IQR , min = Q1 - 1.5*IQ

input = df[col] list của pd.DataFrame
output = list [0,1] thể hiện giá trị nào là outlier

'''

def find_out_IQR_list(col) :
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1    
    print('> {} : IQR range ({},{})'.format(col.name,(Q1 - 1.5 * IQR),(Q3 + 1.5 * IQR)))
    
    return np.where((col < (Q1 - 1.5 * IQR)) | (col > (Q3 + 1.5 * IQR)),1,0)

# 3.Features Processing

## STR/NLP processing
 - https://stackoverflow.com/questions/37011734/pandas-dataframe-str-contains-and-operation


In [0]:
def sum_str(x):
    x = x.split(',')
    l = []
    for i in x:
        if i != "":
            l.append(int(i))
    return sum(l)

In [0]:
t = df[['APE_Net_SumLast3m','Gender','EducationCode','OccupationCode']]
t['APE_Net_SumLast3m_Bins'] = pd.qcut(df.APE_Net_SumLast3m, q=10)
t['OccupationCode_fillna'] =  t.groupby(['Gender','EducationCode','APE_Net_SumLast3m_Bins'])['OccupationCode'].apply(lambda x : x.fillna(x.value_counts().index[0]))

In [0]:
def to_lower_case(item):
    return item.map(lambda x: x if type(x)!=str else x.lower())

def to_upper_case(item):
    return item.map(lambda x: x if type(x)!=str else x.upper())
    
''' example:
train["province"] = to_lower_case(train['province'])
to_upper_case(pd.Series(['abc']))
'''

In [0]:
def replace_data(data,key,values):
  for index, value in enumerate(data):
    if key in str(value):
      data[index] = values
    elif not str(value):
      data[index] = data[index]
  return data
replace_data(train_df["maCv"],'công nhân','CN')
replace_data(train_df["maCv"],'nhân viên','NV')

In [0]:
 t.COMPONENT_CODE.str.contains(r'(OPW[0-9])|(UCW[0-9])|(DSR[0-9])')

## merge/concat
- https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [0]:
df = pd.concat([df1,df2], axis = 0).reset_index(drop=True)

## group by
- http://esantorella.com/2016/06/16/groupby/
- https://towardsdatascience.com/pandas-tips-and-tricks-33bcc8a40bb9


In [0]:
#
# Groupby - Agg join str to str-array.
# Groupby hợp một cột category thành 1 chuỗi category
#
df_gb = df.groupby(["AGCode"], as_index = True).agg({
    "AGLevel":';'.join
    , 'Cnt':(lambda x: ';'.join(x.astype(str)))
    , "YearMonth_Start" : [(lambda x: ';'.join(x.astype(str))),'min']
  
}).reset_index()
df_gb.columns = ['_'.join(col) for col in df_gb.columns]
df_gb.rename(columns={"AGCode_":"AGCode", "AGLevel_join": "AGLevel_flow"
                      ,"Cnt_<lambda>": "Cnt_flow","YearMonth_Start_<lambda_0>":"YearMonth_flow"
                      , "YearMonth_Start_min":"First_month"
                     }, inplace = True)



In [0]:
#
# Conditional groupby, các dạng custom agg của group 
#

sum_unique = (lambda x : sum(x.unique()))
mean_unique = (lambda x : np.mean(x.unique()))
var_unique = (lambda x : np.var(x.unique()))
mode = (lambda x: x.value_counts().index[0])
quantile_mean = lambda x : x[(x>=x.quantile(0.25)) & (x<=x.quantile(0.75))].mean()

t = df[(df.PM_Promo_Time != "Nan") & (df.PM_Promo_Time <= 12)].groupby(["PM_Promo_Time"]).agg({
    ,"Gr_APE_Net_SumNext6m": ['mean','median']
     ,"No_Active_Net_Last3m": ['mean',quantile_mean]

}).reset_index()#.sort_values(by=[('PM_Promo_Time', '')])

t.columns = ['_'.join(col) for col in t.columns]
t

In [0]:
#
# Groupby sử dụng lambda, và các advantage techniques
#

temp = temp.groupby(['Year','month','CHDRNUM','LIFCNUM']).apply(lambda x: sum(x[(x.LIFE == 1)&(x.COMPONENT_ID.isnull()==True)].SUM_ASSURED.fillna(0))).rename('SUM_SA').reset_index()
#.apply(lambda x: sum(x[x.LA_INFO_ID == x.LA_INFO_ID.min()].SUM_ASSURED)) #.SUM_ASSURED.sum()
df = df.merge(temp, on=['Year','month','CHDRNUM','LIFCNUM'])


t = df.groupby(['c','d']).apply(lambda x: sum(x.a+x.b)).rename('e').reset_index()
df.merge(t, on=['c','d'])

# temp = temp.groupby(['Year','month','CHDRNUM','LIFCNUM']).apply(lambda x: sum(x[x.LA_INFO_ID == x.LA_INFO_ID.min()].SUM_ASSURED.fillna(0))).rename('SUM_SA').reset_index()
# #.apply(lambda x: sum(x[x.LA_INFO_ID == x.LA_INFO_ID.min()].SUM_ASSURED)) #.SUM_ASSURED.sum()
# df.merge(temp, on=['c','d'])

**transform**

---
AFAIU I would change the lambda function as follows:
```python
df.groupby('Plate')['LogRatio'].transform(lambda s: s.loc[[True if v < s.quantile(q=0.8) and v > s.quantile(q=0.2) else False for v in s]].mean())
```
This the ```s.loc[]``` accepts an iterable with booleans in order to subset the LogRatio-Series

In order to make it more readable, I'd go for the following solution:
```python
def quartile_subset(logratios,lower,upper):
    # some comment to describe what you are doing
    return logratios.loc[[True if v < logratios.quantile(q=upper) and v > logratios.quantile(q=lower) else False for v in logratios]]

df.groupby('Plate')['LogRatio'].transform(lambda s: quartile_subset(s,0.2,0.8).mean())
```

---
You can use GroupBy + transform with sum twice:
```python
df['e'] = df.groupby(['c', 'd'])[['a', 'b']].transform('sum').sum(1)

print(df)
   a  b  c  d   e
0  1  1  q  z  12
1  2  2  q  z  12
2  3  3  q  z  12
3  4  4  q  o   8
4  5  5  w  o  22
5  6  6  w  o  22
```




### crosstab/ pivot_table
- https://pbpython.com/pandas-crosstab.html


### binning

- https://pbpython.com/pandas-qcut-cut.html


In [0]:
bins = [0, 3, 6,9, 12]
labels = ['1', '2', '3', '4']
t['PM_Promo_Time_bins'] = pd.cut(t['PM_Promo_Time'], bins, labels=labels)

## lag/roll
- https://pandas.pydata.org/pandas-docs/version/0.21.1/generated/pandas.DataFrame.rolling.html
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

In [0]:
t = t.set_index('YearMonth_').groupby(['LIFCNUM_']).rolling(window=365, freq='D',min_periods=1).agg({
    "UAB_PeriodSum":'sum'    
}).reset_index()
t

## sort

## **sort a columns with custom sorter index**
- https://stackoverflow.com/questions/23482668/sorting-by-a-custom-list-in-pandas
- https://stackoverflow.com/questions/13838405/custom-sorting-in-pandas-dataframe
- https://thispointer.com/pandas-sort-rows-or-columns-in-dataframe-based-on-values-using-dataframe-sort_values/

## where/select/query
- https://kanoki.org/2020/01/21/pandas-dataframe-filter-with-multiple-conditions/
- https://stackoverflow.com/questions/49228596/pandas-case-when-default-in-pandas



In [0]:
df1['a'] = np.select(
        [
            df1.AGLevel_flow.str.contains('AG;PM'),      
        ],
        [
           df1.YearMonth_flow.str.split(";").str[1]
        ]
    ,        default = (pd.to_datetime(df1.First_month, format='%Y%m') + pd.DateOffset(months=6)).dt.strftime('%Y%m')
    )

In [0]:
df1['Promo_YearMonth'] = np.select(
        [
            df1.AGLevel_flow.isin(("AG;PM;AG;PM;UM;PM","AG;PM;AG;PM;UM;AG",
                                            "AG;PM;AG;PM;UM;BM","AG;PM;AG;PM;UM;PM;AG","AG;PM;AG;PM;UM"))
            ,df1.AGLevel_flow == "AG;UM"
            ,df1.AGLevel_flow == "AG;PM;AG;PM;AG;PM;UM"
            ,df1.AGLevel_flow.isin(("UM","UM;AG"))
        ],
        [
           df1.YearMonth_flow.str.split(";").str[4]
            ,df1.YearMonth_flow.str.split(";").str[1]
            ,df1.YearMonth_flow.str.split(";").str[6]
            ,df1.YearMonth_flow
        ]
    ,        default = df1.YearMonth_flow.str.split(";").str[2]
    )

In [0]:
query_term = 1
factor_1.query('(`Benefit term` == @query_term)').Factor_others

In [0]:
t['New_Decision'] = np.where(t['LA\'s Decision'].isnull()==False,t['LA\'s Decision']
         ,np.select(
             [
                ( (t.FACTOR5=='YES-UW') | (t.FACTOR3=='YES') | (t.FACTOR4=='YES')  | (t.FACTOR5=='YES')  
                 | (t.FACTOR6=='YES')  | (t.FACTOR7=='YES')  | (t.FACTOR8=='YES') | (t.FACTOR8=='YES-UW') 
                 | (t.FACTOR9=='YES') )                 
             ],
             [
                 'refer UW'                 
             ]
            
         )
        )


## Time

In [0]:
df['Age'] = round((pd.to_datetime(df.Promo_YearMonth, format='%Y%m') - 
                   pd.to_datetime(df.DateOfBirth, format='%Y/%m/%d')) / np.timedelta64(1, 'Y'),0)

df['New_InvoiceDate'] = pd.to_datetime(df.InvoiceDate, format='%d/%m/%Y %H:%M').dt.strftime('%Y%m')

(pd.to_datetime(df_analyze.NewRe_1st_YearMonth_End.fillna(200001), format='%Y%m') 
                                              + pd.DateOffset(months=1)).dt.strftime('%Y%m').astype(int)

# 4.Visualization




## Notes
- [note 1](https://www.kaggle.com/sonannguyenngoc/titanic-survival-prediction-end-to-end-ml-pipeline?scriptVersionId=33324915#Visualizations-&-Analyze), [note 2]()
- [matplotlib guide](https://www.kaggle.com/grroverpr/matplotlib-plotting-guide), [top 50 matplotlib](https://nextjournal.com/sosiristseng/top-50-matplotlib-visualizations#categorical-plots)
- [seaborn guide](https://www.kaggle.com/kralmachine/seaborn-tutorial-for-beginners), [seaborn 1](https://seaborn.pydata.org/tutorial.html)

## Plot continous features
to see:
- distribution
- quantile
- outlier identified by IQR
- distribution & quantile after remove outlier outside IQR




In [0]:

"""Plotting the continous features :    
1. A box plot (or box-and-whisker plot) shows the distribution of quantitative data 
in a way that facilitates comparisons between variables.
2. Distribution graph :to check the linearity of the variables and look 
for skewness of features."""
def plot_continous(cols, df, target=None, info = False, corr = False, drop_3Qplot=False):
    # Using boxplot to analyze the continous feature
    if corr == True:
      df_corr = df.corr().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
      df_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}
                , inplace=True)      

    for col in cols:
        print(f'\n >Column name: {col}')
        plt.figure(figsize=(15,8))
        
        if corr == True:
          print("Top 5 positive correlated w/ featuers \n{}".format(
              df_corr[(df_corr["Feature 1"]==col) & (df_corr["Feature 2"]!=col)].head()))
          print("Top 5 negative correlated w/ featuers \n{}".format(
              df_corr[(df_corr["Feature 1"]==col) & (df_corr["Feature 2"]!=col)].tail()))
        if info == True:

          print("max: {}, min: {}, mean: {}, std: {}, median: {}".format(
              df[col].max(),df[col].min(),
              round(df[col].mean(),3),round(df[col].std(),3),
              df[col].median()))
          print(f"Quantiles | {col}\n",df[col].quantile([.01, .025, .1, .25, .5, .75, .9, .975, .99]))
          print("No unique: {}, No null: {}".format(df[col].nunique(),df[col].isnull().sum()))
          if df[col].nunique() < 10 :
            print(f"Value counts in {col} \n",format(df[col].value_counts()))

        # plot  
        plt.subplot(2, 2, 1)
        if target == None:
            fig = sns.boxplot(col, whis=1.5, data=df)
            fig.legend()
        else:
            fig = sns.boxplot(x=target, y=col, whis=1.5, data=df)
            fig.legend()
        # which defined as the proportion of the IQR past the low and high quartiles to extend the plot whiskers 
        # or interquartile range (IQR)
        # therefore, maximum = Q3 + 1.5*IQR , min = Q1 - 1.5*IQR
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1    
        
        plt.subplot(2, 2, 2)
        fig = sns.distplot(df[col].dropna())#.hist(bins=20)
        fig.set_ylabel('Volumn')
        fig.set_xlabel(col)      
        
        print('No data out of range between Q1 : {} and Q3 : {} = {}' .format(Q1,Q3,((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))).sum()))

        if drop_3Qplot==True:          
          plt.subplot(2, 2, 3)
          tmp = df[(df[col] >= Q1)&(df[col] <= Q3)]#[col]
          if target == None:
              fig = sns.boxplot(col, whis=1.5, data=tmp)
              fig.legend()
          else:
              fig = sns.boxplot(x=target, y=col, whis=1.5, data=tmp)
              fig.legend()
          
          plt.subplot(2, 2, 4)
          fig = sns.distplot(tmp[col])#.dropna())#.hist(bins=20)
          fig.set_ylabel('Volumn')
          fig.set_xlabel(col)   

        plt.show() 

## Plot categorical

In [0]:
def plot_categorical(cols, target, df):
  for col in cols:
    if target == None:
      sns.countplot(x=col, data=df[[col]] ,  palette="Reds_d")      
    else:
      print('Column name: %s' %col)
      sns.countplot(x=col, hue=target, data=df[[col,target]], palette="Reds_d")
      #sns.barplot(x=col, y=target, data=df[[col,target]]
      #        , palette="Reds_d", estimator = sum)
      plt.legend()
    plt.xticks(rotation=90)#-60    
    plt.show()

## 3D plot


Sources:
- https://matplotlib.org/mpl_toolkits/mplot3d/tutorial.html
- https://matplotlib.org/3.1.1/gallery/index.html


Example:
- [how-to-surface-plot-3d-plot-from-dataframe](https://stackoverflow.com/questions/36589521/how-to-surface-plot-3d-plot-from-dataframe)
- [plotting-pandas-crosstab-dataframe-into-3d-bar-chart](https://stackoverflow.com/questions/56336066/plotting-pandas-crosstab-dataframe-into-3d-bar-chart)
- [matplotlib-3d-surface-from-a-rectangular-array-of-heights](https://stackoverflow.com/questions/11766536/matplotlib-3d-surface-from-a-rectangular-array-of-heights)
- [surface-plots-in-matplotlib](https://stackoverflow.com/questions/9170838/surface-plots-in-matplotlib)